<a href="https://www.kaggle.com/code/aisuko/data-cleaning-for-diagnosis-feature?scriptVersionId=195778365" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

Here we do data summarization for icu diagnosis features.
* We summary about mean frequency of medical codes per admission(.mean())
* Total occurrence count od each medical code(.size())
* Fill missing data use `0` (.fillna())

# Load the data

In [1]:
!pip install -U -q polars==1.6.0

In [2]:
import polars as pl

print(pl.__version__)

diag = pl.read_csv("/kaggle/input/reduce-dimension-mimic-iv-v2-0-icu-diagnosis-icd10/dimension_reduce_diag_icu.csv")

1.6.0


In [3]:
import pandas as pd

diag_pd = pd.read_csv("/kaggle/input/reduce-dimension-mimic-iv-v2-0-icu-diagnosis-icd10/dimension_reduce_diag_icu.csv")

In [4]:
diag.describe()

statistic,subject_id,hadm_id,stay_id,new_icd_code
str,f64,f64,f64,str
"""count""",1.291431e6,1.291431e6,1.291431e6,"""1291431"""
"""null_count""",0.0,0.0,0.0,"""0"""
"""mean""",1.4984e7,2.4990e7,3.4993e7,null
"""std""",2.8832e6,2.8808e6,2.8925e6,null
"""min""",1.0000032e7,2.0000094e7,3.0000153e7,"""A01"""
"""25%""",1.2483809e7,2.24909e7,3.2490533e7,null
"""50%""",1.4983584e7,2.500362e7,3.4994611e7,null
"""75%""",1.7486231e7,2.7468749e7,3.7492254e7,null
"""max""",1.9999987e7,2.9999828e7,3.999981e7,"""Z99"""


In [5]:
diag_pd.describe()

,subject_id,hadm_id,stay_id
count,1.291431e+06,1.291431e+06,1.291431e+06
mean,1.498449e+07,2.498995e+07,3.499273e+07
std,2.883164e+06,2.880834e+06,2.892477e+06
min,1.000003e+07,2.000009e+07,3.000015e+07
25%,1.248381e+07,2.249090e+07,3.249053e+07
50%,1.498358e+07,2.500362e+07,3.499461e+07
75%,1.748623e+07,2.746875e+07,3.749225e+07
max,1.999999e+07,2.999983e+07,3.999981e+07


# Group by and count occurrences

In [6]:
# Count occurrences per 'stay_id' and 'new_icd_code'
freq=diag.group_by(['stay_id', 'new_icd_code']).agg(pl.count().alias('mean_frequency'))
freq.head()

/tmp/ipykernel_17/289845405.py:2: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  freq=diag.group_by(['stay_id', 'new_icd_code']).agg(pl.count().alias('mean_frequency'))


stay_id,new_icd_code,mean_frequency
i64,str,u32
36252910,"""K92""",1
35667588,"""R19""",1
31526966,"""G89""",1
30232259,"""I85""",1
34247818,"""Z91""",1


In [7]:
# Count occurrences per 'stay_id' and 'new_icd_code'
freq_pd=diag_pd.groupby(['stay_id','new_icd_code']).size().reset_index(name="mean_frequency")
freq_pd.head()

,stay_id,new_icd_code,mean_frequency
0,30000153,G40,1
1,30000153,J15,1
2,30000153,J93,1
3,30000153,S06,1
4,30000153,S22,1


In [8]:
# Calculate mean frequency per 'new_icd_code'
freq = freq.group_by('new_icd_code').agg(pl.col('mean_frequency').mean().alias('mean_frequency'))
freq.head(2)

new_icd_code,mean_frequency
str,f64
"""A19""",1.0
"""F90""",1.001773


In [9]:
# Calculate mean frequency per 'new_icd_code'
freq_pd=freq_pd.groupby(['new_icd_code'])['mean_frequency'].mean().reset_index()
freq_pd.head(2)

,new_icd_code,mean_frequency
0,A01,1.000000
1,A02,1.035714


In [10]:
# Count total occurrences per 'new_icd_code'
total = diag.group_by('new_icd_code').agg(pl.count().alias('total_count'))
total.head(2)

/tmp/ipykernel_17/2382221481.py:2: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  total = diag.group_by('new_icd_code').agg(pl.count().alias('total_count'))


new_icd_code,total_count
str,u32
"""Z28""",12
"""Q54""",4


In [11]:
total_pd=diag_pd.groupby('new_icd_code').size().reset_index(name="total_count")
total_pd.head(2)

,new_icd_code,total_count
0,A01,1
1,A02,29


# Merge DataFrames

In [12]:
summary = freq.join(total, on='new_icd_code', how='right')
summary.head(2)

mean_frequency,new_icd_code,total_count
f64,str,u32
1.0,"""Z28""",12
1.0,"""Q54""",4


In [13]:
summary_pd=pd.merge(freq_pd,total_pd,on='new_icd_code',how='right')
summary_pd.head(2)

,new_icd_code,mean_frequency,total_count
0,A01,1.000000,1
1,A02,1.035714,29


# Handle Missiong Values

Currently, we use fill null with  0 by using https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html

In [14]:
# Fill missing values with 0
summary = summary.fill_nan(0)
summary.head(2)

mean_frequency,new_icd_code,total_count
f64,str,u32
1.0,"""Z28""",12
1.0,"""Q54""",4


In [15]:
summary_pd=summary_pd.fillna(0)
summary_pd.head(2)

,new_icd_code,mean_frequency,total_count
0,A01,1.000000,1
1,A02,1.035714,29


In [16]:
# Write only 'new_icd_code' column to CSV
# summary.select('new_icd_code').write_csv('./data/summary/diag_features.csv')

summary_pd['new_icd_code'].to_csv('diag_features.csv',index=False)

# Acknowledgements

* https://github.com/SkywardAI/mimic_iv_pipe_sage_maker/blob/main/mainPipeline.ipynb
* https://github.com/SkywardAI/mimic_iv_pipe_sage_maker/blob/1342052a9b5256e58b9fbe5f5a7dfa090cfe38c4/preprocessing/hosp_module_preproc/feature_selection_icu.py#L64
* https://docs.pola.rs/user-guide/getting-started/